In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime
from os import environ
from services.zayev.service_layer.agent import PPOAgent as ZPPO
from matplotlib.dates import relativedelta
import numpy as np


2023-10-12 20:00:24.247423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1800x900 with 0 Axes>

In [14]:

starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
            'database': environ.get("POSTGRES_DB"),
            'user': environ.get("POSTGRES_USER"),
            'password': environ.get("POSTGRES_PASSWORD"),
            'host': environ.get("DB_HOST"),
            'port': environ.get("DB_PORT"),
        }
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 300, 
    "the_current_time_step": starting_time_step,
    "print_output": True,
    "test_steps": 53
} 



zayev = ZPPO(env_config)

/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
print(zayev.Actor.Actor.summary())

In [10]:
zayev.save()

2023-10-12 11:49:11.501124: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_24/kernel/Assign' id:3721 op device:{requested: '', assigned: ''} def:{{{node dense_24/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_24/kernel, dense_24/kernel/Initializer/random_normal)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [11]:
# np.random.seed(np.random.randint(0,10))
zayev.run_batch()
# zayev.run_multiprocesses(num_worker=4)

episode: 1/20, score: 132337.32256902684, average: 68634.40 
episode: 2/20, score: 25654.706715913475, average: 67611.07 
episode: 3/20, score: -323.4446816702548, average: 66031.20 
episode: 4/20, score: 58031.258728599365, average: 65849.38 
episode: 5/20, score: 92217.82535370791, average: 66435.35 
episode: 6/20, score: 33103.95193083165, average: 65710.75 
episode: 7/20, score: 155845.87446590775, average: 67628.52 
episode: 8/20, score: 66831.63146407093, average: 67611.92 
episode: 9/20, score: 52635.2092860578, average: 67306.27 
episode: 10/20, score: 119931.73033799697, average: 68358.78 
episode: 11/20, score: 33576.930873948266, average: 67837.66 
episode: 12/20, score: 34805.30427809722, average: 67578.65 
episode: 13/20, score: 100276.27872124621, average: 67166.98 
episode: 14/20, score: 126244.99164338596, average: 69758.85 
episode: 15/20, score: 87039.51716815635, average: 67768.48 
episode: 16/20, score: 62384.16262783976, average: 68334.04 
episode: 17/20, score: 10

In [15]:
# print(zayev.Actor.Actor)
# [-1.         -0.99999756  0.9822062  -1.         -0.99031776]
# zayev.load()
zayev.test()

2023-10-12 20:07:37.952281: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_29/kernel/Assign' id:1808 op device:{requested: '', assigned: ''} def:{{{node dense_29/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_29/kernel, dense_29/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-10-12 20:07:38.133536: W tensorflow/c/c_api.cc:305] Operation '{name:'lambda_4/SelectV2_1' id:1676 op device:{req


stepping_a [ 0.25 -0.5   0.25 -0.5   0.25  0.25]
[[1.56379204e+02]
 [3.32765632e+00]
 [4.19427938e+02]
 [8.06508246e+03]
 [2.69948462e+04]
 [3.62909287e+03]]
[[  83.29849346 1750.87558874  802.25604172   83.32696147  849.91507752
    83.40747887   86.8         230.41474654]
 [  25.58368408 5246.00036379 2547.92940041   25.59971885 2576.79974602
    25.63920267   25.          800.        ]
 [ 163.74629232 1050.09201815  497.87461227  163.77654739  590.83841064
   163.8357115   158.33333333  126.31578947]
 [  38.79883677  600.42571078  288.58656805   38.81574112  278.88837551
    38.88674565   37.5         533.33333333]
 [  20.90841334 1751.68870569  934.45576153   20.93477174  927.24739874
    20.95200131   21.33333333  937.5       ]]
[0]
change_in_shares for index: 0 301.64633472212626 937.5 802.2560417206539
change_in_shares for index: 1 -1965.6122347637995 937.5 2547.9294004051712
change_in_shares for index: 2 153.5657889208838 937.5 497.8746122746549
change_in_shares for index: 3 -

In [13]:
print(zayev.previous_state)
print(zayev.previous_actn)
print(zayev.latest_state)

[array([[[1.62674585e+02, 1.10860679e+04, 7.61565479e+02, 1.62686290e+02,
         7.73503775e+02, 1.62703075e+02, 1.64358313e+02, 9.16915063e+02],
        [2.13185567e+01, 6.57769817e+03, 2.47946306e+03, 2.13390126e+01,
         2.85236432e+03, 2.13537738e+01, 2.10116718e+01, 1.00000000e+00],
        [2.57325798e+02, 1.40404644e+03, 5.72872717e+02, 2.57329014e+02,
         5.95154415e+02, 2.57354023e+02, 2.59453132e+02, 1.00000000e+00],
        [5.39192741e+01, 7.96764736e+02, 2.72898402e+02, 5.39218209e+01,
         3.00957130e+02, 5.39252069e+01, 5.38027045e+01, 1.00000000e+00],
        [2.18657237e+01, 3.02063070e+03, 8.50623935e+02, 2.18724618e+01,
         9.08496472e+02, 2.19197656e+01, 2.22115352e+01, 9.37500000e+02]]]), array([[[1.87556726e+02],
        [3.16982984e+00],
        [4.16509321e+02],
        [8.49581413e+03],
        [2.54566517e+04],
        [4.50531000e+03]]]), array([[0]])]
[ 0.33333334 -0.3333333  -0.3333333  -0.3333333   0.33333334  0.33333334]
[array([[[1.63

In [35]:
ztt = zayev.latest_state
stt_stt = zayev.latest_state[0][0]
print(stt_stt)

total_potfolio = zayev.latest_state[2][0]
for st_dta in stt_stt:
    total_potfolio = total_potfolio + st_dta[0]*st_dta[7]
print(total_potfolio)
print(zayev.env.the_current_time_step - relativedelta(hours=100))

[[8.19678646e+01 2.43880864e+03 9.75345014e+02 8.22008843e+01
  7.66613268e+02 8.23515861e+01 7.68176859e+01 6.51296015e+02]
 [2.62161971e+01 5.64381844e+03 2.32921692e+03 2.62239934e+01
  2.42789306e+03 2.62451703e+01 2.66008026e+01 1.00000000e+00]
 [1.70331760e+02 1.14572909e+03 5.41098317e+02 1.70358625e+02
  5.98110364e+02 1.70411557e+02 1.74174741e+02 1.65967199e+02]
 [6.12547799e+01 6.54401212e+02 2.76779553e+02 6.12573036e+01
  3.42052660e+02 6.12590837e+01 6.11700756e+01 1.00000000e+00]
 [2.21214677e+01 2.15738819e+03 9.18469113e+02 2.21449470e+01
  9.28346293e+02 2.22346375e+01 2.13678578e+01 9.37500000e+02]]
[102481.17548137]
1994-12-30 08:00:00+00:00


In [8]:
tm = starting_time_step + relativedelta(hours=25)
print(tm)

1995-01-02 11:00:00


In [6]:
zayev.latest_state

[array([[[  86.98892013, 1780.88718796,  806.78320262,   86.99418474,
           815.32319724,   87.01223627,   86.22798652,  231.66143027],
         [  24.55947431, 5311.76025676, 2294.7676268 ,   24.5622656 ,
          2544.55969405,   24.56600397,   24.61056939,  795.63826776],
         [ 139.2253307 , 1086.44600003,  505.13989644,  139.25715658,
           561.13738873,  139.33339698,  144.20254547,  126.31578947],
         [  38.81905216,  614.39851888,  328.47856092,   38.82553138,
           308.16662239,   38.84516044,   38.48886214,  533.33333333],
         [  21.18280565, 1786.27803173,  976.56194102,   21.18713267,
           841.46566394,   21.19766482,   21.08023049,  937.5       ]]]),
 array([[[1.71481016e+02],
         [3.49922248e+00],
         [4.32156117e+02],
         [7.54711667e+03],
         [2.72181363e+04],
         [3.93169616e+03]]]),
 array([[0]])]

In [8]:
actp = np.array([1,2,3])*1000
print(actp)

[1000 2000 3000]


In [13]:
log_std = .5 * np.ones(5, dtype=np.float32)
std = np.exp(log_std)

print(log_std)
print(std)

[0.5 0.5 0.5 0.5 0.5]
[1.6487212 1.6487212 1.6487212 1.6487212 1.6487212]


In [6]:
# print(zayev.env.the_current_time_step)

1995-01-16 20:00:00+00:00
